In [12]:
import pandas as pd
import faiss

import tensorflow_hub as hub

from pprint import pprint

In [2]:
# IMPORTANT - used to find doc index
tokens_chunk_size = 512

In [15]:
legal_docs = pd.read_csv("../data/legal_text_classification.csv")
index = faiss.read_index("non_chunked_court_text.index")

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"

model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [21]:
query1 = "claims related to shoe companies"
#query1 = "claims involving large sums of money"

# Encode the query into an embedding
query_embedding = embed([query1]).numpy()

# Search for similar embeddings in the index
D, I = index.search(query_embedding, k=5)  # Retrieve the top 5 similar documents

# Print the indices and distances of the similar documents
print("Top 5 similar documents:")
indices = []
for i, d in zip(I[0], D[0]):
    print(f"Index: {i}, Similarity Score: {1 - d}")
    indices.append(i)

Top 5 similar documents:
Index: 18197, Similarity Score: -0.35356605052948
Index: 14894, Similarity Score: -0.42606401443481445
Index: 10145, Similarity Score: -0.47372210025787354
Index: 14491, Similarity Score: -0.4985116720199585
Index: 22965, Similarity Score: -0.5052845478057861


In [22]:
idx = 18197

pprint (legal_docs.iloc[idx]["case_title"])
pprint (legal_docs.iloc[idx]["case_text"])

('Elderslie Finance Corporation Ltd v Newpage (No 6) [2007] FCA 1030 ; (2007) '
 '160 FCR 423')
('Court may approve a compromise if one of a number of ways of presenting a '
 'claim involves a "debt" claim: Elderslie Finance Corporation Ltd v Newpage '
 '(No 6) [2007] FCA 1030 ; (2007) 160 FCR 423 at [40] - [41] . In the present '
 'case, one way in which EBD has framed its claim is that payments made by it '
 'to EBM upon false invoices were made pursuant to sham transactions, and of '
 'no legal effect and that consequently, there is no basis upon which EBM may '
 'retain the money paid by EBD. This is a claim for restitution of an amount '
 'wrongly paid which is capable of constituting a "debt". Discretion')
